#### 1. 학습데이터 확인 (finance_data)
###### 1) NLP 학습/검증 데이터
###### 2) Nvidia 뉴스/주식 데이터
###### 3) + 주가예측모델용 데이터


#### 2. NLP 모델 확인 (finbert)
###### 모델구현


#### 3. 학습 및 성능 검증


#### 4. 주가예측모델 구축


#### 5. 감성 feature 추가 후 비교 

In [3]:
import pandas as pd
import os
import numpy as np
import sys
import os
import argparse
import shutil
import random
from pathlib import Path
import matplotlib.pyplot as plt


from tqdm import tqdm 
tqdm.pandas()

import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

c:\Users\KOSA\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
!pip install torch

  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl (228.5 MB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl (3.5 MB)
  Using cached tbb-2021.13.0-py3-none-win_amd64.whl (286 kB)


You should consider upgrading via the 'c:\users\hangil\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [5]:
!pip install transformers

You should consider upgrading via the 'c:\users\hangil\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [9]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("kakaobank/kf-deberta-base")
tokenizer = AutoTokenizer.from_pretrained("kakaobank/kf-deberta-base")

text = "카카오뱅크와 에프엔가이드가 금융특화 언어모델을 공개합니다."
tokens = tokenizer.tokenize(text)
print(tokens)

inputs = tokenizer(text, return_tensors="pt")
model_output = model(**inputs)
print(model_output)

c:\Users\hangil\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\hangil\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hangil\.cache\huggingface\hub\models--kakaobank--kf-deberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as 

['카카오', '##뱅크', '##와', '에프', '##엔', '##가이', '##드', '##가', '금융', '##특', '##화', '언어', '##모델', '##을', '공개', '##합니다', '.']
BaseModelOutput(last_hidden_state=tensor([[[ 0.3266,  1.0502, -0.6938,  ...,  1.5691, -1.0747,  1.1697],
         [-0.3798,  2.1233,  0.3641,  ..., -0.1863, -0.9263, -0.7637],
         [ 0.6740,  1.2344, -0.6436,  ..., -0.5377, -1.3844,  0.1552],
         ...,
         [ 1.3960, -0.5337, -1.5322,  ..., -0.4973,  0.5771, -0.4824],
         [ 1.7078, -0.1235, -1.4388,  ...,  1.7612,  0.1432, -1.7462],
         [ 0.2516,  0.9185, -0.9216,  ...,  1.8321, -1.4633,  0.3245]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


In [23]:
news_df = pd.read_csv("./data/nvidia_sentimental_analisis.csv")
stock_df = pd.read_csv("./data/nvidia_stock_2015_to_2024.csv", index_col=0)

In [24]:
news_df['Summary']

,Date,Summary,sentiment_rf,sentiment_nn
0,2024-06-14,As bolsas de Nova York fecharam sem direção ún...,neutral,positive
1,2024-06-14,Os futuros dos índices de ações dos EUA tivera...,positive,positive
2,2024-06-14,Os acionistas da Tesla (NASDAQ:TSLA) mostraram...,neutral,neutral
3,2024-06-13,"Em um movimento recente, Mark A. Stevens, dire...",neutral,neutral
4,2024-06-13,(Bloomberg) -- A Broadcom experimentou uma alt...,neutral,neutral
...,...,...,...,...
906,2016-10-24,Investing.com – E.U.A. - As ações fecharam em ...,negative,neutral
907,2016-08-12,Investing.com – E.U.A. - As ações fecharam ina...,negative,neutral
908,2016-05-13,Investing.com – E.U.A. - As ações fecharam em ...,negative,neutral
909,2015-11-06,Investing.com – E.U.A. - As ações fecharam ina...,negative,neutral


In [25]:
stock_df

,date,open,high,low,close,adjclose,volume
0,2015-01-02,0.503250,0.507000,0.495250,0.503250,0.483218,113680000
1,2015-01-05,0.503250,0.504750,0.492500,0.494750,0.475056,197952000
2,2015-01-06,0.495500,0.496000,0.479250,0.479750,0.460654,197764000
3,2015-01-07,0.483250,0.487500,0.477000,0.478500,0.459453,321808000
4,2015-01-08,0.484000,0.499500,0.483750,0.496500,0.476737,283780000
...,...,...,...,...,...,...,...
2364,2024-05-24,104.448997,106.474998,103.000000,106.469002,106.460258,429494000
2365,2024-05-28,110.244003,114.939003,109.883003,113.901001,113.891647,652728000
2366,2024-05-29,113.050003,115.491997,110.901001,114.824997,114.815567,557442000
2367,2024-05-30,114.650002,115.819000,109.663002,110.500000,110.490921,487350000
